In [34]:
from pulp import *
import pandas as pd
import numpy as np

In [35]:
A5 = LpProblem("hornby",LpMinimize)

In [36]:
df_cost = pd.read_excel("Exhibits.xlsx")

In [37]:
df_cost = pd.read_excel("Exhibits.xlsx",sheet_name="cost")
df_demand = pd.read_excel("Exhibits.xlsx",sheet_name="demand")
df_dis_cost = pd.read_excel("Exhibits.xlsx",sheet_name="dis_cost")
df_cost.head()

,W/H,Location,Fixed cost,Variable cost,Min_A,Max_A
0,1,Atlanta,2700,6,200,1750
1,2,Buffalo,2900,8,150,1250
2,3,Chicago,3500,9,250,2000
3,4,Cincinnati,2200,7,200,1500
4,5,Detroit,3300,8,200,1750


In [38]:
df_dis_cost.set_index("Unnamed: 0",inplace=True)

In [39]:
from_cities = df_cost.Location.tolist()

In [40]:
from_cities = df_cost.Location.tolist()#8
to_cities = df_demand.Location.tolist()#22

fixed_cost = df_cost["Fixed cost"].tolist()
fixed_cost = dict(zip(from_cities,fixed_cost))

fixed_cost

{'Atlanta': 2700,
 'Buffalo': 2900,
 'Chicago': 3500,
 'Cincinnati': 2200,
 'Detroit': 3300,
 'Pittsburgh': 3000,
 'Richmond': 2000,
 'St. Louis': 1800}

In [41]:
df_cost["Min_A"]

0    200
1    150
2    250
3    200
4    200
5    200
6    150
7    200
Name: Min_A, dtype: int64

In [42]:
df_dis_cost.values.transpose()

array([[ 1,  4, 13,  7,  4,  2, 16,  9, 13, 10, 15,  9, 14, 12,  4,  8,
         8,  6, 16, 11,  8, 12],
       [13, 15,  1,  6, 10, 11, 10,  6,  3,  4,  4, 10,  8,  9,  9,  8,
        14, 12, 11,  2,  7, 13],
       [16, 15, 10, 11, 15, 14,  1,  7,  7,  8,  6,  7,  2,  4, 12,  8,
        12, 11,  2,  9, 14,  7],
       [11, 12,  8,  6, 10,  9,  9,  3,  6,  4,  8,  6,  7,  6,  7,  4,
        10,  8, 10,  6,  9,  9],
       [15, 16,  4,  8, 13, 13,  6,  6,  3,  5,  1,  9,  4,  7, 11,  8,
        14, 12,  8,  5, 11, 11],
       [11, 13,  4,  6,  8,  9,  9,  6,  5,  4,  7,  8,  9,  8,  9,  8,
        12, 10, 11,  3,  7, 11],
       [ 8, 11,  7,  4,  4,  7, 14,  7,  8,  7, 11, 10, 12, 11,  6,  8,
        12,  9, 15,  5,  1, 14],
       [12, 10, 13, 10, 14, 11,  7,  7, 10,  9, 11,  4,  6,  4,  9,  6,
         6,  7,  5, 11, 14,  1]])

In [43]:
var_cost = df_cost["Variable cost"].tolist()
var_cost = dict(zip(from_cities,var_cost))

Min_A = df_cost["Min_A"].tolist()
Min_A = dict(zip(from_cities,Min_A))

Max_A = df_cost["Max_A"].tolist()
Max_A = dict(zip(from_cities,Max_A))

dis_cost = makeDict([from_cities,to_cities],\
                    df_dis_cost.values.transpose().tolist())

In [44]:
demand = df_demand.Volume.tolist()
demand = dict(zip(to_cities,demand))
# Decision Variable
open_or_not = LpVariable.dicts("x",from_cities,lowBound=0,cat="Binary")
product = LpVariable.dicts("p",(from_cities,to_cities), lowBound =0, cat="Continuous")
over_min= LpVariable.dicts("m",(from_cities),lowBound=0,cat="Continuous")

In [45]:
#Objective Function
variable_dis_cost=lpSum([dis_cost[i][j] * product[i][j] for i in from_cities for j in to_cities])
variable_fixed_cost=lpSum([fixed_cost[i] * open_or_not[i] for i in from_cities])
variable_var_cost=lpSum([over_min[i] * var_cost[i] for i in from_cities])
A5 += variable_dis_cost + variable_fixed_cost + variable_var_cost


In [46]:
#Constraints
# products should be smaller than Max_A
for i in from_cities:
    A5 += lpSum([product[i][j] for j in to_cities]) <= open_or_not[i]*Max_A[i]
# Products should satisfy the demand    
for j in to_cities:
    A5 += lpSum([product[i][j] for i in from_cities])>=demand[j]
# For each factory,over_min plus min_A should more than or equal to production amount  
for i in from_cities:
    A5 += (over_min[i] + Min_A[i]) >= lpSum(product[i][j] for j in to_cities)
                                             

In [47]:
A5.writeLP("HornBy.lp")
A5.solve()
print("Status:",LpStatus[A5.status])
print(value(A5.objective))

Status: Optimal
56075.0


In [48]:
output=[]
for i in from_cities:# 8 rows
    var_output=[]
    for j in to_cities: # 22 columns
        var_output.append(product[i][j].varValue)
    var_output.append(open_or_not[i].varValue)
    output.append(var_output)
to_cities.append('Rent')
output_df = pd.DataFrame(output,index=from_cities, columns=to_cities)

In [50]:
output_df.transpose()

,Atlanta,Buffalo,Chicago,Cincinnati,Detroit,Pittsburgh,Richmond,St. Louis
Atlanta,275.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Birmingham,160.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Buffalo,0.0,240.0,0.0,0.0,0.0,0.0,0.0,0.0
Charteston,0.0,0.0,0.0,0.0,0.0,0.0,260.0,0.0
Charlotte,135.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chattanooga,160.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chicago,0.0,0.0,0.0,0.0,0.0,0.0,0.0,400.0
Cincinnati,0.0,0.0,0.0,200.0,0.0,0.0,0.0,0.0
Cleveland,0.0,320.0,0.0,0.0,0.0,0.0,0.0,0.0
Columbus,0.0,0.0,0.0,220.0,0.0,0.0,0.0,0.0
